In [3]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/u189511/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
training_data = pd.read_csv('train_file.csv')

In [5]:
training_data

,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1,-1,-1,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1,-1,-1,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1,-1,-1,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1,-1,-1,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1,-1,-1,0.000000,0.141084
...,...,...,...,...,...,...,...,...,...,...,...
55927,jQ3CeLRCb9,Fidel Castro Lashes Out at Obama After Cuba Visit,Retired Cuban leader Fidel Castro slammed Pres...,Wall Street Journal,obama,2016-03-29 01:35:06,794,10,5,-0.135417,-0.055902
55928,akNYeJ8opY,JOHN CRISP | Obama's strategic reaction to Bru...,President Obama caught some predictable flak f...,Kitsap Sun,obama,2016-03-29 01:35:08,0,0,0,0.236228,0.056110
55929,n2DGs0c8IG,Think Trump's 45 Percent Tariffs Are Bad? Try ...,While Trump wants to put large tariffs on impo...,Huffington Post,obama,2016-03-29 01:35:09,102,4,0,0.025747,0.114820
55930,P0EBiaSEjq,Microsoft finally releases giant Surface,Microsoft’s business customers are finally beg...,TechEye,microsoft,2016-03-29 01:38:00,0,0,0,0.000000,-0.028296


In [6]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55932 entries, 0 to 55931
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDLink             55932 non-null  object 
 1   Title              55932 non-null  object 
 2   Headline           55932 non-null  object 
 3   Source             55757 non-null  object 
 4   Topic              55932 non-null  object 
 5   PublishDate        55932 non-null  object 
 6   Facebook           55932 non-null  int64  
 7   GooglePlus         55932 non-null  int64  
 8   LinkedIn           55932 non-null  int64  
 9   SentimentTitle     55932 non-null  float64
 10  SentimentHeadline  55932 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 4.7+ MB


In [7]:
training_data.isna().sum()

IDLink                 0
Title                  0
Headline               0
Source               175
Topic                  0
PublishDate            0
Facebook               0
GooglePlus             0
LinkedIn               0
SentimentTitle         0
SentimentHeadline      0
dtype: int64

In [8]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55932 entries, 0 to 55931
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDLink             55932 non-null  object 
 1   Title              55932 non-null  object 
 2   Headline           55932 non-null  object 
 3   Source             55757 non-null  object 
 4   Topic              55932 non-null  object 
 5   PublishDate        55932 non-null  object 
 6   Facebook           55932 non-null  int64  
 7   GooglePlus         55932 non-null  int64  
 8   LinkedIn           55932 non-null  int64  
 9   SentimentTitle     55932 non-null  float64
 10  SentimentHeadline  55932 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 4.7+ MB


In [9]:
sample = pd.read_csv('sample.csv')
sample

,IDLink,SentimentTitle,SentimentHeadline
0,CZLVHanbC2,0.000000,-0.014113
1,VFkR6aHMVJ,0.080799,-0.038613
2,P7iEhVkGlM,-0.045645,-0.120476
3,ij5NwOmhCO,0.291667,0.094653
4,AZhpvyzDfC,0.291663,0.094658
5,K68BWTk8nY,0.044194,0.058465
6,m95WZhuovK,0.000000,0.326718
7,4qwM4lsvYm,0.043212,-0.088388
8,u8iGT7DY2a,-0.079057,0.083853
9,3clx8GUpzY,0.000000,0.063732


In [10]:
training_data = training_data[training_data['Facebook'].notna()]



In [11]:
training_data.isna().sum()

IDLink                 0
Title                  0
Headline               0
Source               175
Topic                  0
PublishDate            0
Facebook               0
GooglePlus             0
LinkedIn               0
SentimentTitle         0
SentimentHeadline      0
dtype: int64

In [12]:
training_data = training_data.dropna()


In [13]:
training_data.isna().sum()

IDLink               0
Title                0
Headline             0
Source               0
Topic                0
PublishDate          0
Facebook             0
GooglePlus           0
LinkedIn             0
SentimentTitle       0
SentimentHeadline    0
dtype: int64

In [14]:
test_data = pd.read_csv('test_file.csv')
test_data
source = list(training_data['Source'])
topic = list(training_data['Topic'])
publishDate = list(training_data['PublishDate'])
facebook = list(training_data['Facebook'])
googlePlus = list(training_data['GooglePlus'])
linkedIn = list(training_data['LinkedIn'])


In [15]:
sentimentTitle = list(training_data['SentimentTitle'])
sentimentHeadline = list(training_data['SentimentHeadline'])

training_sentimentTitle = sentimentTitle[0:40575]
testing_sentimentTitle = sentimentTitle[40575:]

training_sentimentHeadline = sentimentHeadline[0:40575]
testing_sentimentHeadline = sentimentHeadline[40575:]

title = list(training_data['Title'])
headline = list(training_data['Headline'])

len(max(headline, key=len))

training_title = title[0:40575]
testing_title = title[40575:]

training_headline = headline[0:40575]
testing_headline = headline[40575:]


In [32]:
tokenizer = Tokenizer(num_words=100000, oov_token="<OOV>")
# tokenizer.fit_on_texts(training_title)
# tokenizer.fit_on_texts(training_headline)

# word_index = tokenizer.word_index

# training_titleSequences = tokenizer.texts_to_sequences(training_title)
# training_paddedTitle = pad_sequences(training_titleSequences, maxlen=170, padding='post', truncating='post')

# testing_titleSequences = tokenizer.texts_to_sequences(testing_title)
# testing_paddedTitle = pad_sequences(testing_titleSequences, maxlen=170, padding='post', truncating='post')

# training_headlineSequences = tokenizer.texts_to_sequences(training_headline)
# training_paddedHeadline = pad_sequences(training_headlineSequences, maxlen=446, padding='post', truncating='post')

# testing_headlineSequences = tokenizer.texts_to_sequences(testing_headline)
# testing_paddedHeadline = pad_sequences(testing_headlineSequences, maxlen=446, padding='post', truncating='post')

# training_paddedTitle = np.array(training_paddedTitle)
# testing_paddedTitle = np.array(testing_paddedTitle)
# training_paddedHeadline = np.array(training_paddedHeadline)
# testing_paddedHeadline = np.array(testing_paddedHeadline)

In [33]:
tokenizer.fit_on_texts(title)
tokenizer.fit_on_texts(headline)
tokenizer.fit_on_texts(topic)

word_index = tokenizer.word_index
    
titleSequences = tokenizer.texts_to_sequences(title)
#170
paddedTitle = pad_sequences(titleSequences, maxlen=None, padding='post', truncating='post')

headlineSequences = tokenizer.texts_to_sequences(headline)
# paddedHeadline = pad_sequences(headlineSequences, maxlen=446, padding='post', truncating='post')
paddedHeadline = pad_sequences(headlineSequences, maxlen=None, padding='post', truncating='post')

sourceSequences = tokenizer.texts_to_sequences(source)
paddedSource = pad_sequences(sourceSequences, maxlen=None, padding='post', truncating='post')

topicSequences = tokenizer.texts_to_sequences(topic)
paddedTopic = pad_sequences(topicSequences, maxlen=None, padding='post', truncating='post')

paddedTitle = np.array(paddedTitle)
paddedHeadline = np.array(paddedHeadline)
paddedTopic = np.array(paddedTopic)
paddedSource = np.array(paddedSource)

In [20]:
print(type(paddedHeadline[0][0]))
print(paddedSource[0])
print(paddedTopic[0])

<class 'numpy.int32'>
[4193  119    0    0    0    0    0    0    0    0    0]
[9]


In [ ]:
for x in [paddedTitle, paddedHeadline, source, topic, publishDate, facebook, googlePlus, linkedIn, sentimentTitle, sentimentHeadline]:
    print(len(x))

In [18]:
unified_attribute_data = [np.array(x) for x in zip(paddedTitle, paddedHeadline, paddedSource, paddedTopic, facebook, googlePlus, linkedIn)]
print(len(unified_attribute_data))
unified_target_data = [np.array(x) for x in zip(sentimentTitle, sentimentHeadline)]
print(len(unified_target_data))
unified_attribute_data = np.array(unified_attribute_data)
unified_target_data = np.array(unified_target_data)

In [27]:
title_words = [x.split() for x in title]
title_words
headline_words = [x.split() for x in headline]

stop_words = set(stopwords.words('english'))
num_value = tokenizer.texts_to_sequences(headline_words[0][0])
print(headline_words[0][0])
print(tokenizer.word_index[(headline_words[0][0]).lower()])
# print(tokenizer.word_index)
num_value

Obama
9


[[2990], [1944], [6], [963], [6]]

In [43]:
# title_words = [x.split() for x in title]
# title_words
# headline_words = [x.split() for x in headline]
# headline_words[0][0]

import re
title_words = [re.findall(r"[\w']+", x) for x in title]
headline_words = [re.findall(r"[\w']+", x) for x in headline]


In [41]:
# stop_words = set(stopwords.words('english'))

# modified_data = list()
# attribute_data = list()
# target_data = list()
# for a in range(0, 55757):
#     for b in range(0, len(headline_words[a])):
#         if (headline_words[a][b]).lower() not in stop_words:
#             # modified_data.append([headline_words[a][b], source[a], topic[a], publishDate[a], facebook[a], googlePlus[a], linkedIn[a]])
#             for c in range(0, len(title_words[a])):
#                 if (title_words[a][c]).lower() not in stop_words:
#                     num_headline = tokenizer.word_index[(headline_words[a][b]).lower()]
#                     num_title = tokenizer.word_index[(title_words[a][c]).lower()]
#                     num_topic = tokenizer.word_index[(topic[a]).lower()]
#                     modified_data.append([num_title, num_headline, num_topic, facebook[a], googlePlus[a], linkedIn[a], sentimentTitle[a], sentimentHeadline[a]])
#                     attribute_data.append([num_title, num_headline, num_topic, facebook[a], googlePlus[a], linkedIn[a]])
#                     target_data.append([sentimentTitle[a], sentimentHeadline[a]])
                           
# # SOURCE IS DELETED
# print(attribute_data[0])
# print(target_data[0])

In [44]:
stop_words = set(stopwords.words('english'))

num_modified_data = list()
num_attribute_data = list()
num_target_data = list()
for a in range(0, 55757):
    for b in range(0, len(headlineSequences[a])):
        for c in range(0, len(titleSequences[a])):
            num_topic = tokenizer.word_index[(topic[a]).lower()]
            num_modified_data.append([titleSequences[a][c], headlineSequences[a][b], num_topic, facebook[a], googlePlus[a], linkedIn[a], sentimentTitle[a], sentimentHeadline[a]])
            num_attribute_data.append([titleSequences[a][c], headlineSequences[a][b], num_topic, facebook[a], googlePlus[a], linkedIn[a]])
            num_target_data.append([sentimentTitle[a], sentimentHeadline[a]])
                           
# SOURCE IS DELETED
print(num_attribute_data[0])
print(num_target_data[0])

[5, 5, 5, -1, -1, -1]
[0.0, -0.0533001790889026]


In [45]:
from sklearn import model_selection

x_train, x_test, y_train, y_test = model_selection.train_test_split(np.array(num_attribute_data[:40000]), np.array(num_target_data[:40000]), random_state=42, test_size = 0.3)

In [46]:

print(len(x_train))
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)

28000
(28000, 6)
(28000, 2)
(12000, 2)


In [47]:
# model.add(Dense(10, input_dim=5, kernel_initializer='uniform', activation='tanh'))
#     model.add(Dropout(0.15))
#     model.add(Dense(7, kernel_initializer='uniform', activation='relu'))
#     model.add(Dropout(0.15))
#     model.add(Dense(3,kernel_initializer='uniform', activation='sigmoid'))
    

model = tf.keras.Sequential([
    tf.keras.layers.Dense(24,input_dim=6, activation='tanh'),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                168       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 26        
Total params: 494
Trainable params: 494
Non-trainable params: 0
_________________________________________________________________


In [48]:
# training_attribute_data =  np.array(attribute_data[:5362151])

num_epochs = 10

history = model.fit(x_train, y_train, 
                    epochs=num_epochs,
                    validation_data=(x_test, y_test),
                    verbose=2)

Train on 28000 samples, validate on 12000 samples
Epoch 1/10
28000/28000 - 2s - loss: -2.3154e-01 - acc: 0.1320 - val_loss: -9.1066e-01 - val_acc: 0.1346
Epoch 2/10
28000/28000 - 2s - loss: -3.6112e+00 - acc: 0.1352 - val_loss: -7.9851e+00 - val_acc: 0.1346
Epoch 3/10
28000/28000 - 2s - loss: -1.5477e+01 - acc: 0.1352 - val_loss: -2.4519e+01 - val_acc: 0.1346
Epoch 4/10
28000/28000 - 2s - loss: -3.5869e+01 - acc: 0.1352 - val_loss: -4.9680e+01 - val_acc: 0.1346
Epoch 5/10
28000/28000 - 2s - loss: -6.4195e+01 - acc: 0.1352 - val_loss: -8.1089e+01 - val_acc: 0.1346
Epoch 6/10
28000/28000 - 2s - loss: -9.8756e+01 - acc: 0.1352 - val_loss: -1.1878e+02 - val_acc: 0.1346
Epoch 7/10
28000/28000 - 2s - loss: -1.3628e+02 - acc: 0.1352 - val_loss: -1.6016e+02 - val_acc: 0.1346
Epoch 8/10
28000/28000 - 2s - loss: -1.8201e+02 - acc: 0.1351 - val_loss: -2.0711e+02 - val_acc: 0.1345
Epoch 9/10
28000/28000 - 2s - loss: -2.3028e+02 - acc: 0.1351 - val_loss: -2.5864e+02 - val_acc: 0.1344
Epoch 10/10
28